# Download
First we need to download all files from the server and combine them into a single dataframe

In [ ]:
import pandas as pd
import os
files = os.listdir('data')
print(files)
df = pd.concat([pd.read_csv('data' + file) for file in files], ignore_index=True)

['part3.csv', 'part4.csv', 'part5.csv']


FileNotFoundError: [Errno 2] No such file or directory: 'part3.csv'

# Mean times for each setting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_avg_time_by_setting(df):
    # Compute average time for each setting
    avg_times = df.groupby(["ui_mode", "environment"])["time_to_finish"].mean().reset_index()
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x="ui_mode", y="time_to_finish", hue="environment", data=avg_times)

    # Labels and title
    plt.xlabel("UI Mode")
    plt.ylabel("Average Time to Finish (ms)")
    plt.title("Average Task Completion Time by UI Mode and Environment")
    plt.legend(title="Environment")

    # Show the plot
    plt.show()

# Call the function
plot_avg_time_by_setting(df)

# Mean times for trial

In [ ]:
def plot_overall_learning_trend(df):
    # Select only relevant trials
    selected_trials = list(range(1, 61, 1))
    df_filtered = df[df["trial"].isin(selected_trials)]

    # Compute the mean time per trial across all settings
    avg_times = df_filtered.groupby("trial")["time_to_finish"].mean().reset_index()

    # Plot the trend
    plt.figure(figsize=(10, 6))
    sns.lineplot(x="trial", y="time_to_finish", data=avg_times, marker="o", linewidth=2)

    # Labels and title
    plt.xlabel("Trial Number")
    plt.ylabel("Average Time to Finish (ms)")
    plt.title("Overall Learning Trend: Task Completion Time Over Trials")

    # Show the plot
    plt.show()

# Call the function
plot_overall_learning_trend(df)